In [ ]:
import numpy as np  
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import PolynomialFeatures
from itertools import product
import pandas as pd
import gradio as gr

def solve_ridge(Cxx, Cxy, regparam):
    return torch.linalg.solve(Cxx + regparam * torch.eye(Cxx.shape[0]), Cxy[:,None]).squeeze()

In [ ]:
def process(n_train, degree, sig=1):
    X_train = torch.randn(n_train, 1)

    # Sort the data
    X_train, _ = torch.sort(X_train, dim=0)

    # Create a polynomial feature transformer
    poly = PolynomialFeatures(degree=degree)

    # Create polynomial features
    X = poly.fit_transform(X_train)
    X = torch.tensor(X, dtype=torch.float32)

    theta_tru = torch.tensor([1, -1, 1], dtype=torch.float32)*1
    theta_tru = torch.cat([theta_tru, torch.zeros(X.shape[1] - theta_tru.shape[0])])

    f_tru = X @ theta_tru
    y_train = f_tru + torch.randn(n_train) * sig
    
    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_subplot(111)

    Cxx = X.T @ X / n_train # Covariance of X
    Cxy = X.T @ y_train / n_train # Cross covariance of X and y
    theta_ridge = solve_ridge(Cxx, Cxy, regparam = 0)

    f_ridge = X @ theta_ridge
     # Plot the data
    ax.scatter(X_train, y_train, s=1)
    ax.plot(X_train, f_tru, color='red')
    ax.plot(X_train, f_ridge, color='green')

    return fig

demo = gr.Interface(fn=process, 
                    inputs=[
                        gr.Number(value = 500, precision=0),
                        gr.Slider(2, 30, value=5, step=1, label="degree"),
                        gr.Slider(0, 10, value=1, step=0.25, label="noise s.d.")
                    ], 
                    outputs="plot", live=True)
demo.launch()

In [ ]:
def show_mse(n_train, degree, sig, lamax):
    X_train = torch.randn(n_train, 1)

    # Sort the data
    X_train, _ = torch.sort(X_train, dim=0)

    # Create a polynomial feature transformer
    poly = PolynomialFeatures(degree=degree)

    # Create polynomial features
    X = poly.fit_transform(X_train)
    X = torch.tensor(X, dtype=torch.float32)

    # standardize the data
    X = (X - X.mean(dim=0)) / X.std(dim=0)
    X[:,0] = 1

    theta_tru = torch.tensor([1, -1, 1], dtype=torch.float32)*1
    theta_tru = torch.cat([theta_tru, torch.zeros(X.shape[1] - theta_tru.shape[0])])

    f_tru = X @ theta_tru
   
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111)

    Cxx = X.T @ X / n_train # Covariance of X

    n_reps = 50
    n_regparams = 100
    regparams = np.logspace(-4, lamax, n_regparams)
    res = []
    for rep, regparam in product(range(n_reps), regparams):
        y_train = f_tru + torch.randn(n_train) * sig
        
        Cxy = X.T @ y_train / n_train
        theta_ridge = solve_ridge(Cxx, Cxy, regparam)
        f_ridge = X @ theta_ridge
        # loss = torch.mean((f_ridge - y_train)**2).item()
        loss = torch.mean((f_tru - f_ridge)**2).item()
        # store rep, regparam, loss
        res.append({'rep':rep, 'regparam':regparam, 'loss':loss})

    res = pd.DataFrame(res)

    # average over repetitions
    res = res.groupby('regparam').mean().reset_index()

    # plot the results
    ax.plot(res.regparam, res.loss)
    #plt.xscale('log')

    # svd of X
    U, S, V = torch.linalg.svd(X / np.sqrt(n_train), full_matrices=False)
    U.shape, S.shape, V.shape

    xi_tru = U.T @ f_tru / np.sqrt(n_train)
    mu = S**2


    def get_theor_mse(regparam):
        Gamma = torch.diag(mu / (mu + regparam))
        z = (torch.eye(Gamma.shape[0]) - Gamma) @ xi_tru
        bias = (z**2).sum()
        variance = (sig**2 / n_train) * torch.trace(Gamma**2)
        return bias + variance

    # theor_mses = torch.zeros_like(regparams)
    ax.plot(regparams, [get_theor_mse(regparam) for regparam in regparams])


    return fig

demo = gr.Interface(fn=show_mse, 
                    inputs=[
                        gr.Number(value = 500, precision=0),
                        gr.Slider(2, 30, value=5, step=1, label="degree"),
                        gr.Slider(0, 10, value=1, step=0.1, label="noise s.d."),
                        gr.Slider(-3, 2, value=-1, step=0.25, label="max log lambda")
                    ], 
                    outputs="plot", live=True, allow_flagging = False)
demo.launch()